# Rock Paper Scissors Competition

The goal is to make a routine which beats other routines in rock-paper-scissors competition.
Following the convention established in this competition, we say the event
$\textbf{ROCK}$ is 0, $\textbf{PAPER}$ is 1, and $\textbf{SCISSORS}$ is 2.
Given two random variables $X$ and $Y$ on the space 
$\Omega = \{\textbf{ROCK}, \textbf{PAPER}, \textbf{SCISSORS}\}$ or alternatively,
$\Omega = \{0, 1, 2\}$,
we say 
1. $X$ beats $Y$ in a round if $Y + 1 \equiv X \mod 3$;
2. $X$ loses $Y$ in a round if $Y - 1 \equiv X \mod 3$;
3. $X$ ties $Y$ in a round if $Y = X$.

The round is scored by 
$$s(X, Y) = \begin{cases} 1 & Y + 1 \equiv X \mod 3 \\ -1 & Y - 1 \equiv X \mod 3 \\ 0 & X = Y \end{cases} \, .$$


A game is between two agents is then a sequence of $N$ pairs random variables
$\{(X_{t}, Y_{t})\}_{1 \leq t \leq N}$ whose total score is $S = \sum_t s(X_t, Y_t)$.
Agent $X$ wins if the total score is positive and agent $Y$ wins if the total score is negative.

As hinted in *configuration.signs*, this game can be generalize to an $M$ point space.

We evaluate a strategy on the distribution of the random variable $S$. 
We would like to maximize $\mathrm{E}[S]$ and minimize $\mathrm{VAR}[S]$ 
for $X$ given a collection of strategies $\{Y\}$.
Let us expand on this by formulating different strategies.

We say $X_t$ is uniformly randomly chosen if $P(X_t=0) = 1/3$, $P(X_t=1) = 1/3$, and $P(X_t=2) = 1/3$.
... et cetera. You get it.

Another strategy is the deterministic reactive strategy in which the next move is determined entirely by the opponents previous move.
For this next example, we would prefer to represent 
$\textbf{ROCK}$, $\textbf{PAPER}$, $\textbf{SCISSORS}$ 
as vectors $[1, 0, 0]$, $[0, 1, 0]$, and $[0, 0, 1]$ respectively.
Similarly we can represent probabilities of $X_t$ as a vector $P(X_t) := [P(X_t=0), P(X_t=1), P(X_t=2)]$ which is subject to the condition that each entry is nonnegative and the sum is $1$.
The reactive strategy which picks $X_t=0$ if $Y_{t-1}=2$, $X_t=1$ if $Y_{t-1}=0$, and $X_t=2$ if $Y_{t-1}=1$ can be explicitly written as 
$$X_t = \begin{bmatrix} 0 & 0 & 1 \\ 1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix} Y_{t-1} \, .$$

We can now formulate our goal for this project. 
The opponent has some strategy determined by the game state $(X_1, \dots, X_{t-1}, Y_1, \dots, Y_{t-1})$
$$P(Y_t) = f(X_1, \dots, X_{t-1}, Y_1, \dots, Y_{t-1})$$ and the counterstrategy is
to set $X_t = \arg \max P(Y_t) + 1 \mod 3$.
Thus if we can formulate a prediction of $Y_t$, we can beat $Y$.

The strategy here assumes $Y_t$ is a less general than written above, namely it is 
of the form
$$P(Y_t) = \sum_{1 \leq j \leq p} C_{j} Y_{t - j} + \sum_{1 \leq k \leq q} D_{k} X_{t - k} + \varepsilon \, .$$
This form covers the deterministic reactive strategy, the uniform strategy (via $\varepsilon = [1/3, 1/3, 1/3]$),
and the constant strategies.

This model is contained in the `RegressiveDecision` class.
Technical details are the refitting the data is done at increments of `RegressiveDecision.update_model`
and the training only takes `RegressiveDecision.model_memory` most recent examples.
This allows the strategy to adapt to "discontinuity" in the opponents strategy.

In [2]:
import numpy as np
import pandas as pd
import sklearn.linear_model
import sklearn.ensemble


class RegressiveDecision():
    """
    Uses time series classification model to reactively
    choose output.
    """
    
    def __init__(self, 
                 reg_type=sklearn.ensemble.RandomForestClassifier,
                 p=3, 
                 q=2,
                 model_memory=25, 
                 memory_buffer=2000, 
                 update_period=10,
                 M=3,
    ):
        """
        Constructor.
        
        Args:
            reg_type (class): Classifier class that fits Scikit-learn API.
                Must have ${fit} and ${predict} methods.
            p (int): Length of sequence of opponent's moves used in prediction.
            q (int): Length of sequence of model's moves used in prediction.
            model_memory (int): Length of sequence used in training.
            memory_buffer (int): Length of memory pre-allocated.
            update_period (int): Period of retraining model.
            M (int): Number of game actions -- 3 for rock, paper, scissors.
            
        Returns:
            None: Constructor.
        """
        self.M = M
        self.p = p
        self.q = q
        self.reg_type = reg_type
        self.p_cols = [f"p_{i}" for i in range(p)]
        self.q_cols = [f"q_{i}" for i in range(q)]
        self.opp_col = ["opp"]
        self.act_col = ["act"]
        self.columns = self.opp_col + self.p_cols + self.act_col + self.q_cols
        self.memory_buffer = memory_buffer
        self.model_memory = model_memory
        memory = np.zeros(shape=(memory_buffer, q+p+2))
        memory[:max(p,q)] = np.random.randint(M, size=(max(p,q), q+p+2))
        self.memory = pd.DataFrame(memory, columns=self.columns, dtype=int)
        self.update_period = update_period
        self.index = 0
        self.reg = None
        
        
    def predict(self):
        """
        Predict
        """
        ret_val = 0
        lb = (self.index - self.model_memory 
            if self.index >= self.model_memory else 0
        )
        if self.index < self.p:
            ret_val = self.memory.at[self.index, self.opp_col[0]].astype(int)
        elif np.all(
            self.memory.loc[lb:self.index, self.opp_col[0]]
                ==self.memory.loc[lb, self.opp_col[0]]
        ):
            ret_val = self.memory.at[lb, self.opp_col[0]].astype(int)
        elif ((self.index<self.update_period) 
              or (self.index%self.update_period==0)
              or (self.reg is None)
        ):
            self.reg = self.reg_type()
            X = self.memory.loc[lb:self.index, self.p_cols+self.q_cols]
            Y = self.memory.loc[lb:self.index, self.opp_col[0]]
            self.reg.fit(X, Y)
            ret_val = self.reg.predict(
                [self.memory.loc[self.index, self.p_cols+self.q_cols].values]
            ).astype(int)
        else:
            ret_val = self.reg.predict(
                [self.memory.loc[self.index, self.p_cols+self.q_cols].values]
            ).astype(int)
        ret_val = int(ret_val+1) % self.M
        self.memory.at[self.index, self.act_col[0]] = ret_val
        return ret_val
    
    
    def update(self, val):
        """
        Update info of opponent's move.
        """
        self.memory.at[self.index, self.opp_col[0]] = int(val)
        if self.index >= self.p:
            self.memory.loc[self.index+1, self.p_cols] = (
                self.memory.loc[self.index, self.opp_col + self.p_cols[:-1]]
                    .values
            )
        else:
            self.memory.loc[self.index+1, self.p_cols[:self.index+1]] = (
                self.memory.loc[range(self.index+1), self.opp_col[0]].values
            )
        if self.index >= self.q:
            self.memory.loc[self.index+1, self.q_cols] = (
                self.memory.loc[self.index, self.act_col + self.q_cols[:-1]]
                    .values
            )
        else:
            self.memory.loc[self.index+1, self.q_cols[:self.index+1]] = (
                self.memory.loc[range(self.index+1), self.act_col[0]].values
            )
        self.index += 1

    
agent = None
#run_reg_type = sklearn.linear_model.LogisticRegression
run_reg_type = sklearn.ensemble.RandomForestClassifier
def regressive_decision_wrapper(observation, configuration):
    """
    Entry point for this competition's application.
    """
    global agent
    if agent is None:
        agent = RegressiveDecision(
            reg_type=run_reg_type,
            M=configuration.signs,
        )
    else:
        agent.update(observation.lastOpponentAction)
    return agent.predict()

In [2]:
pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_columns = None

DIM = 3

In [3]:
import enum

class RPS(enum.Enum):
    ROCK = 0
    SCISSORS = 1
    PAPER = 2

In [5]:
Id = np.diag(np.ones(DIM, dtype=int))

class MarkovStrategy():

    def __init__(self):
        self.p = 2
        self.mem = [np.random.randint(DIM) for _ in range(self.p)]
        self.C = np.random.normal(size=(self.p, DIM, DIM))

    def gen(self):
        while True:
            xt = np.array([Id[x] for x in reversed(self.mem[-self.p:])])
            x_f = (np.tensordot(xt, self.C, axes=([0, 1], [0, 1]))
               + np.random.normal(size=(DIM))
            )
            t = np.argmax(x_f)
            self.mem.append(t)
            return t
        

strat = None 
def Markov(observation, configuration):
    global strat
    if strat is None:
        strat = MarkovStrategy(M=configuration.signs)
    return int(strat.gen())

In [6]:
def Rock():
    while True:
        yield RPS.ROCK.value

def Scissors():
    while True:
        yield RPS.SCISSORS.value
    
def Paper():
    while True:
        yield RPS.PAPER.value
    
x = [np.random.randint(DIM)]
def Forward():
    while True:
        act = (x[-1] + 1) % DIM
        x.append(act)
        yield act
    
y = [np.random.randint(DIM)]
def Backward():
    while True:
        act = (x[-1] - 1) % DIM
        x.append(act)
        yield act    
    

In [7]:
def reactive_wrapper(observation, configuration):
    return (
        (observation.lastOpponentAction + 1) % configuration.signs
            if observation.step > 0
            else np.random.randint(configuration.signs)
    )                   

In [8]:
class Observation():
    def __init__(self):
        pass

observation = Observation()
observation.lastOpponentAction = None
observation.step = 0 

configuration = Observation()
configuration.signs = 3

In [9]:
runs = 1000

p1 = []
p2 = []

#agent2 = Forward()
#agent2 = Rock()
#agent2 = Backward()
agent2 = Markov()

for i in range(runs):
    if len(p2) > 0:
        observation.lastOpponentAction = p2[-1]
    val1 = regressive_decision_wrapper(observation, configuration)
    if len(p1) > 0:
        observation.lastOpponentAction = p1[-1]
    #val2 = reactive_wrapper(observation, configuration)
    val2 = next(agent2)
    observation.step += 1
    p1.append(val1)
    p2.append(val2)

In [10]:
df = pd.DataFrame({"Agent1": p1, "Agent2": p2})

In [11]:
opp = "Agent2"
mod = "Agent1"
df["Win"] = (df[opp]+1)%DIM==df[mod]
df["Tie"] = (df[opp]+0)%DIM==df[mod]
df["Lose"] = (df[opp]-1)%DIM==df[mod]

df["Win"].sum(), df["Tie"].sum(), df["Lose"].sum(),

(506, 217, 277)